In [0]:
carrier = spark.read.parquet("/mnt/bronze/carrier/carrier_claims.parquet")

In [0]:
from pyspark.sql.functions import to_date, col

def standardize_columns(df):
    return df.toDF(*[c.lower() for c in df.columns])
    
# Clean Carrier

carrier_clean = (
    carrier.dropDuplicates(["DESYNPUF_ID", "CLM_ID"])
    .withColumn("CLM_FROM_DT", to_date(col("CLM_FROM_DT"), "yyyyMMdd"))
    .withColumn("CLM_THRU_DT", to_date(col("CLM_THRU_DT"), "yyyyMMdd"))
    .fillna("unknown")
)

carrier_clean = standardize_columns(carrier_clean)

In [0]:
carrier_clean.write.mode("overwrite").parquet("/mnt/silver/carrier/carrier_clean")

In [0]:
from pyspark.sql.functions import col, year, when, to_date, lit

# Carrier Silver Table
carrier_silver = carrier_clean.filter(
    col("clm_id").isNotNull() & col("clm_from_dt").isNotNull()
).select(
    "desynpuf_id", "clm_id", "clm_from_dt", "clm_thru_dt",
    "icd9_dgns_cd_1", "prf_physn_npi_1", "hcpcs_cd_1"
)

In [0]:
carrier_silver.write.mode("overwrite").parquet("/mnt/silver/carrier/carrier_silver")

In [0]:
carrier_clean.show(5)